In [2]:
import pandas as pd

In [2]:
#import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

data = pd.read_csv("mnist_train.csv")
ytrain = data["label"].values.astype(np.int)

Xtrain = data.drop(columns="label" , inplace=False).values
Xtrain = scaler.fit_transform(Xtrain)

test = pd.read_csv("mnist_test.csv")
ytest = test["label"].values
Xtest = test.drop(columns="label" , inplace=False).values
Xtest = scaler.transform(Xtest)

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "tf_logs/run-{}".format(now)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [3]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_classes = 10

#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/")

In [4]:
def layer(X , n_neurons  , name , activation = None):
    n = int(X.get_shape()[1])
    stddev = 2/np.sqrt(n_neurons + n)
    with tf.name_scope(name=name):
        W = tf.Variable(tf.truncated_normal(shape=(n , n_neurons) , stddev=stddev) , name="W" , dtype=tf.float32)
        b = tf.Variable(tf.zeros( [n_neurons] ) , name="b" , dtype=tf.float32)
        z = tf.add(tf.matmul(X , W) , b)
        if activation is not None :
            return activation(z)
        else :
            return z

In [4]:
X = tf.placeholder(tf.float32 , shape=(None , n_inputs) , name="X")
y = tf.placeholder(tf.int64 , shape=(None) , name="y")

he_init = tf.variance_scaling_initializer()

with tf.name_scope("DNN") :
    hidden1 = tf.layers.dense(X , n_hidden1 , activation=tf.nn.elu , name="hidden1"  , kernel_initializer=he_init , kern) # can be repaced with tf.layers.dense
    hidden2 = tf.layers.dense(hidden1 , n_hidden2 , activation=tf.nn.elu , name="hidden2" , kernel_initializer=he_init)
    logits = tf.layers.dense(hidden2 , n_classes , name="outputs" )
    
with tf.name_scope("loss") :
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y , logits=logits)
    loss = tf.reduce_mean(xentropy , name="Loss")

with tf.name_scope("Records") :
    loss_summary = tf.summary.scalar("Loss_sum" , loss)
    file_writer = tf.summary.FileWriter(log_dir , tf.get_default_graph())
    
    
lr = 0.01
with tf.name_scope("train") :
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits , y , 1)
    accuracy = tf.reduce_mean(tf.cast(correct , dtype=tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def fetch_data(step , batch_size , n_batches , Xm , ym , perms):
    
    starting = step * batch_size
    indicies = perms[ starting:starting+batch_size]
    return Xm[indicies , :] , ym[indicies].reshape(-1)

In [6]:
n_epoch = 15
n_batches = 1000
batch_size = 50
perms = np.random.permutation(batch_size * n_batches)

with tf.Session() as sess: 
    sess.run(init)
    
    for epoch in range(n_epoch):
        for step in range(n_batches):
            #indicies = perms[ step*batch_size:step*batch_size+batch_size]
            Xbatch , ybatch = Xtrain[step*batch_size:step*batch_size+batch_size , :] , ytrain[step*batch_size:step*batch_size+batch_size].reshape(-1)
            sess.run(training_op , feed_dict={X:Xbatch , y:ybatch})
            sum_ = loss_summary.eval(feed_dict={X:Xbatch , y:ybatch})
            file_writer.add_summary(sum_ , step + epoch * n_batches)
        acc_train = sess.run(accuracy , feed_dict={X:Xtrain , y:ytrain})
        print("Epoch : {} Train_acc : {} ".format(epoch , acc_train))
    saver.save(sess , "/tmp/mnist_dnn_plain.ckpt")

Epoch : 0 Train_acc : 0.9175166487693787 
Epoch : 1 Train_acc : 0.934066653251648 
Epoch : 2 Train_acc : 0.9433000087738037 
Epoch : 3 Train_acc : 0.9504833221435547 
Epoch : 4 Train_acc : 0.9563000202178955 
Epoch : 5 Train_acc : 0.9606500267982483 
Epoch : 6 Train_acc : 0.9641333222389221 
Epoch : 7 Train_acc : 0.9673666954040527 
Epoch : 8 Train_acc : 0.9700833559036255 
Epoch : 9 Train_acc : 0.9726333618164062 
Epoch : 10 Train_acc : 0.9744166731834412 
Epoch : 11 Train_acc : 0.9763333201408386 
Epoch : 12 Train_acc : 0.9775166511535645 
Epoch : 13 Train_acc : 0.9788333177566528 
Epoch : 14 Train_acc : 0.9800166487693787 


In [7]:
with tf.Session() as sess:
    saver.restore(sess , "/tmp/mnist_dnn_plain.ckpt")
    test_acc = accuracy.eval(feed_dict={X:Xtest , y:ytest})
    print("Test Accuracy : {}" .format(test_acc))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/mnist_dnn_plain.ckpt
Test Accuracy : 0.9660000205039978
